In [1]:

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.text import *
from fastai.utils.mem import GPUMemTrace #call with mtrace

In [3]:
import sklearn.feature_extraction.text as sklearn_text
import pickle

In [ ]:
# path = untar_data(URLs.IMDB_SAMPLE)

# Get csv

In [10]:
path = Path('/home/quantran/.fastai/data/imdb_sample')

In [11]:
path.ls()

[PosixPath('/home/quantran/.fastai/data/imdb_sample/texts.csv')]

In [12]:

df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


# use fastai lib to tokenize

In [75]:
%%time
# throws `BrokenProcessPool' Error sometimes. Keep trying `till it works!

count = 0
error = True
while error:
    try: 
        # Preprocessing steps
        movie_reviews = (TextList.from_csv(path, 'texts.csv', cols='text')
                         .split_from_df(col=2)
                         .label_from_df(cols=0))
        error = False
        print(f'failure count is {count}\n')    
    except: # catch *all* exceptions
        # accumulate failure count
        count = count + 1
        print(f'failure count is {count}')

failure count is 0

CPU times: user 152 ms, sys: 64.7 ms, total: 216 ms
Wall time: 1.23 s


In [76]:
type(movie_reviews)

fastai.data_block.LabelLists

In [97]:
movie_reviews.valid

LabelList (200 items)
x: TextList
xxbos xxmaj this very funny xxmaj british comedy shows what might happen if a section of xxmaj london , in this case xxmaj xxunk , were to xxunk itself independent from the rest of the xxup uk and its laws , xxunk & post - war xxunk . xxmaj merry xxunk is what would happen . 
 
  xxmaj the explosion of a wartime bomb leads to the xxunk of ancient xxunk which show that xxmaj xxunk was xxunk to the xxmaj xxunk of xxmaj xxunk xxunk ago , a small historical xxunk long since forgotten . xxmaj to the new xxmaj xxunk , however , this is an unexpected opportunity to live as they please , free from any xxunk from xxmaj xxunk . 
 
  xxmaj stanley xxmaj xxunk is excellent as the minor city xxunk who suddenly finds himself leading one of the world 's xxunk xxunk . xxmaj xxunk xxmaj margaret xxmaj xxunk is a delight as the history professor who sides with xxmaj xxunk . xxmaj others in the stand - out cast include xxmaj xxunk xxmaj xxunk , xxmaj paul xxmaj xxunk , x

In [78]:
len(movie_reviews.train),len(movie_reviews.valid)

(800, 200)

In [79]:
type(movie_reviews.train),type(movie_reviews.train.x)

(fastai.data_block.LabelList, fastai.text.data.TextList)

In [80]:
movie_reviews.train.y

CategoryList (800 items)
negative,positive,negative,positive,negative
Path: /home/quantran/.fastai/data/imdb_sample

In [81]:
movie_reviews.train.x[0],type(movie_reviews.train.x[0])

(Text xxbos xxmaj un - xxunk - believable ! xxmaj meg xxmaj ryan does n't even look her usual xxunk lovable self in this , which normally makes me forgive her shallow xxunk acting xxunk . xxmaj hard to believe she was the producer on this dog . xxmaj plus xxmaj kevin xxmaj kline : what kind of suicide trip has his career been on ? xxmaj xxunk ... xxmaj xxunk ! ! ! xxmaj finally this was directed by the guy who did xxmaj big xxmaj xxunk ? xxmaj must be a replay of xxmaj jonestown - hollywood style . xxmaj xxunk !,
 fastai.text.data.Text)

In [82]:
movie_reviews.train.x[0].data #tokenized

array([   2,    5, 4619,   25, ...,   10,    5,    0,   52])

Vocab

In [83]:
for i,(k,v) in enumerate(movie_reviews.vocab.stoi.items()):
    print(f'{k}: {v}')
    if i==20: break

xxunk: 0
xxpad: 1
xxbos: 2
xxeos: 3
xxfld: 4
xxmaj: 5
xxup: 6
xxrep: 7
xxwrep: 8
the: 9
.: 10
,: 11
and: 12
a: 13
of: 14
to: 15
is: 16
it: 17
in: 18
i: 19
that: 20


In [84]:
movie_reviews.vocab.itos[:30]

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 '.',
 ',',
 'and',
 'a',
 'of',
 'to',
 'is',
 'it',
 'in',
 'i',
 'that',
 'this',
 '"',
 "'s",
 '\n \n ',
 '-',
 'was',
 'as',
 'for',
 'movie']

In [85]:
movie_reviews.vocab.stoi['\n \n ']

24

In [86]:
len(movie_reviews.vocab.stoi),len(movie_reviews.vocab.itos)
# not equal because a lot of 'unknown words' aka words with freq <= 3 (or sth)
# are mapped to same idx in stoi (which is 0)

(19161, 6008)

In [87]:
temp = [[] for i in range(9)]
temp

[[], [], [], [], [], [], [], [], []]

In [88]:
for word, num in movie_reviews.vocab.stoi.items():
    if num in list(range(9)):
        temp[num].append(word)

In [89]:
temp[0][0]

'xxunk'

In [90]:
temp[1:]

[['xxpad'],
 ['xxbos'],
 ['xxeos'],
 ['xxfld'],
 ['xxmaj'],
 ['xxup'],
 ['xxrep'],
 ['xxwrep']]

In [91]:
len(temp[0]) - 1

13153

In [92]:
len(temp[0]) - 1 == len(movie_reviews.vocab.stoi) - len(movie_reviews.vocab.itos)

True

# Writer bow from scratch

In [98]:

TokenCounter = lambda review_index : Counter((movie_reviews.train.x)[review_index].data)
TokenCounter(0).items()

dict_items([(2, 1), (5, 15), (4619, 1), (25, 3), (0, 8), (867, 1), (52, 5), (3776, 1), (1800, 1), (95, 1), (37, 1), (85, 1), (191, 1), (64, 2), (935, 1), (2738, 1), (517, 1), (18, 1), (21, 3), (11, 1), (84, 1), (2417, 1), (192, 1), (88, 1), (3777, 1), (1801, 1), (127, 1), (10, 3), (269, 1), (15, 1), (273, 1), (73, 1), (26, 2), (9, 2), (1360, 1), (35, 2), (1213, 1), (1144, 1), (1145, 1), (2418, 1), (91, 1), (63, 1), (245, 1), (14, 2), (1361, 1), (1447, 1), (65, 1), (40, 1), (796, 1), (103, 1), (72, 2), (99, 1), (534, 1), (616, 1), (48, 1), (282, 1), (54, 1), (90, 1), (219, 1), (228, 1), (43, 1), (13, 1), (3778, 1), (3779, 1), (355, 1), (492, 1)])

# Create embedding vectors for bow

This is just a (# of reviews) x (# of vocabs) embedding matrix, each row has len(vocab) entry with only few nonzero entries (which are counters) 

=> sparse matrix

In [102]:
n_terms = len(movie_reviews.vocab.itos)
n_docs = len(movie_reviews.train.x)

In [103]:
make_token_counter = lambda review_index: Counter(movie_reviews.train.x[review_index].data)

In [104]:
def count_vectorizer(review_index,n_terms = n_terms,make_token_counter = make_token_counter):
    # input: review index, n_terms, and tokenizer function
    # output: embedding vector for the review
    embedding_vector = np.zeros(n_terms)        
    keys = list(make_token_counter(review_index).keys())
    values = list(make_token_counter(review_index).values())
    embedding_vector[keys] = values
    return embedding_vector

In [105]:
# make the embedding vector for the first review
embedding_vector = count_vectorizer(0)

In [107]:
len(embedding_vector)

6008